In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
import time

from utils.sgd import *

from utils.utils import normalizeRows, softmax, sigmoid
from utils.treebank import StanfordSentiment


In [2]:
def naiveSoftmaxLossAndGradient(
    centerWordVec,
    outsideWordIdx,
    outsideVectors,
    dataset
):
    """ Naive Softmax loss & gradient function for word2vec models

    Implementation of  the naive softmax loss and gradients between a center word's 
    embedding and an outside word's embedding. This will be the building block
    for our word2vec models. 

    Arguments:
    centerWordVec -- numpy ndarray, center word's embedding
                    in shape (word vector length, )
                    (v_c in the pdf handout)
    outsideWordIdx -- integer, the index of the outside word
                    (o of u_o in the pdf handout)
    outsideVectors -- outside vectors is
                    in shape (num words in vocab, word vector length) 
                    for all words in vocab (tranpose of U in the pdf handout)
    dataset -- needed for negative sampling, unused here.

    Return:
    loss -- naive softmax loss
    gradCenterVec -- the gradient with respect to the center word vector
                     in shape (word vector length, )
                     (dJ / dv_c in the pdf handout)
    gradOutsideVecs -- the gradient with respect to all the outside word vectors
                    in shape (num words in vocab, word vector length) 
                    (dJ / dU)
    """

    softmax_vect = softmax(outsideVectors @ centerWordVec)
    loss = -np.log(softmax_vect[outsideWordIdx])

    gradCenterVec = - outsideVectors[outsideWordIdx] + outsideVectors.T @ softmax_vect

    gradOutsideVecs = softmax_vect.reshape(-1, 1) @ centerWordVec.reshape(1, -1)
    gradOutsideVecs[outsideWordIdx] = (softmax_vect[outsideWordIdx] - 1) * centerWordVec # w != o
    

    
    return loss, gradCenterVec, gradOutsideVecs

In [3]:
def getNegativeSamples(outsideWordIdx, dataset, K):
    """ Samples K indexes which are not the outsideWordIdx """

    negSampleWordIndices = [None] * K
    for k in range(K):
        newidx = dataset.sampleTokenIdx()
        while newidx == outsideWordIdx:
            newidx = dataset.sampleTokenIdx()
        negSampleWordIndices[k] = newidx
    return negSampleWordIndices

In [7]:
def negSamplingLossAndGradient(
    centerWordVec,
    outsideWordIdx,
    outsideVectors,
    dataset,
    K=10
):
    """ Negative sampling loss function for word2vec models

    Implementation of  the negative sampling loss and gradients for 
    a centerWordVec and a outsideWordIdx word vector as a building block
    for word2vec models. K is the number of negative samples to take.

    Note: The same word may be negatively sampled multiple times. 

    Arguments/Return Specifications: same as naiveSoftmaxLossAndGradient
    """

    negSampleWordIndices = getNegativeSamples(outsideWordIdx, dataset, K)
    indices = [outsideWordIdx] + negSampleWordIndices

    scores = outsideVectors[indices] @ centerWordVec # (K+1,)
    sigmoid_neg_score_ks = sigmoid(-scores[1:]).reshape(-1, 1) # (K, 1)

    loss = -np.log(sigmoid(scores[0])) - np.sum(np.log(sigmoid_neg_score_ks))

    tmp_k = (1 - sigmoid_neg_score_ks) * outsideVectors[negSampleWordIndices] # (K, d)
    gradCenterVec = (sigmoid(scores[0]) - 1) * outsideVectors[outsideWordIdx] \
        + np.sum(tmp_k, axis=0) # (d,)

    gradOutsideVecs = np.zeros_like(outsideVectors) # (V, d)
    gradOutsideVecs[outsideWordIdx] = (sigmoid(scores[0]) - 1) * centerWordVec # (d,)
    # Count for duplications:
    for i, k in enumerate(negSampleWordIndices):
        gradOutsideVecs[k] += (1 - sigmoid_neg_score_ks[i]) * centerWordVec # (d, )

    ### END YOUR CODE

    return loss, gradCenterVec, gradOutsideVecs




In [8]:
def skipgram(currentCenterWord, windowSize, outsideWords, word2Ind,
             centerWordVectors, outsideVectors, dataset,
             word2vecLossAndGradient=naiveSoftmaxLossAndGradient):
    """ Skip-gram model in word2vec

    Implement the skip-gram model in this function.

    Arguments:
    currentCenterWord -- a string of the current center word
    windowSize -- integer, context window size
    outsideWords -- list of no more than 2*windowSize strings, the outside words
    word2Ind -- a dictionary that maps words to their indices in
              the word vector list
    centerWordVectors -- center word vectors (as rows) is in shape 
                        (num words in vocab, word vector length) 
                        for all words in vocab (V in pdf handout)
    outsideVectors -- outside vectors is in shape 
                        (num words in vocab, word vector length) 
                        for all words in vocab (transpose of U in the pdf handout)
    word2vecLossAndGradient -- the loss and gradient function for
                               a prediction vector given the outsideWordIdx
                               word vectors, could be one of the two
                               loss functions you implemented above.

    Return:
    loss -- the loss function value for the skip-gram model
            (J in the pdf handout)
    gradCenterVec -- the gradient with respect to the center word vector
                     in shape (num words in vocab, word vector length)
                     (dJ / dv_c in the pdf handout)
    gradOutsideVecs -- the gradient with respect to all the outside word vectors
                    in shape (num words in vocab, word vector length) 
                    (dJ / dU)
    """

    loss = 0.0
    gradCenterVecs = np.zeros(centerWordVectors.shape)
    gradOutsideVectors = np.zeros(outsideVectors.shape)

    ### YOUR CODE HERE (~8 Lines)

    centerWordVec = centerWordVectors[word2Ind[currentCenterWord]]
        
    for outsideWord in outsideWords:
        loss_j, gradCenterVecs_j, gradOutsideVectors_j = word2vecLossAndGradient(
            centerWordVec, word2Ind[outsideWord], outsideVectors, dataset)
        
        loss += loss_j
        gradCenterVecs[word2Ind[currentCenterWord]] += gradCenterVecs_j
        gradOutsideVectors += gradOutsideVectors_j


    ### END YOUR CODE
    
    return loss, gradCenterVecs, gradOutsideVectors


In [ ]:
def word2vec_sgd_wrapper(word2vecModel, word2Ind, wordVectors, dataset,
                         windowSize,
                         word2vecLossAndGradient=naiveSoftmaxLossAndGradient):
    batchsize = 50
    loss = 0.0
    grad = np.zeros(wordVectors.shape)
    N = wordVectors.shape[0]
    centerWordVectors = wordVectors[:int(N/2),:]
    outsideVectors = wordVectors[int(N/2):,:]
    for i in range(batchsize):
        windowSize1 = random.randint(1, windowSize)
        centerWord, context = dataset.getRandomContext(windowSize1)

        c, gin, gout = word2vecModel(
            centerWord, windowSize1, context, word2Ind, centerWordVectors,
            outsideVectors, dataset, word2vecLossAndGradient
        )
        loss += c / batchsize
        grad[:int(N/2), :] += gin / batchsize
        grad[int(N/2):, :] += gout / batchsize

    return loss, grad

In [10]:
random.seed(314)
dataset = StanfordSentiment()
tokens = dataset.tokens()
nWords = len(tokens)

In [11]:
# We are going to train 10-dimensional vectors for this assignment
dimVectors = 10

# Context size
C = 5

In [12]:
startTime=time.time()
wordVectors = np.concatenate(
    ((np.random.rand(nWords, dimVectors) - 0.5) /
       dimVectors, np.zeros((nWords, dimVectors))),
    axis=0)
wordVectors = sgd(
    lambda vec: word2vec_sgd_wrapper(skipgram, tokens, vec, dataset, C,
        negSamplingLossAndGradient),
    wordVectors, 0.3, 40000, None, True, PRINT_EVERY=10)
# Note that normalization is not called here. This is not a bug,
# normalizing during training loses the notion of length.

print("sanity check: cost at convergence should be around or below 10")
print("training took %d seconds" % (time.time() - startTime))

NameError: name 'sgd' is not defined

In [ ]:
wordVectors = np.concatenate(
    (wordVectors[:nWords,:], wordVectors[nWords:,:]),
    axis=0)

visualizeWords = [
    "great", "cool", "brilliant", "wonderful", "well", "amazing",
    "worth", "sweet", "enjoyable", "boring", "bad", "dumb",
    "annoying", "female", "male", "queen", "king", "man", "woman", "rain", "snow",
    "hail", "coffee", "tea"]

visualizeIdx = [tokens[word] for word in visualizeWords]
visualizeVecs = wordVectors[visualizeIdx, :]
temp = (visualizeVecs - np.mean(visualizeVecs, axis=0))
covariance = 1.0 / len(visualizeIdx) * temp.T.dot(temp)
U,S,V = np.linalg.svd(covariance)
coord = temp.dot(U[:,0:2])

for i in range(len(visualizeWords)):
    plt.text(coord[i,0], coord[i,1], visualizeWords[i],
        bbox=dict(facecolor='green', alpha=0.1))

plt.xlim((np.min(coord[:,0]), np.max(coord[:,0])))
plt.ylim((np.min(coord[:,1]), np.max(coord[:,1])))

plt.savefig('word_vectors.png')